In [32]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import html5lib
import lxml.html as lh

In [162]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r=requests.get(URL)  #gets the code from the wikipedia page


Takes the code from the page and pareses through, giving an html version of the page to read.

In [163]:
soup = BeautifulSoup(r.content, 'html.parser') 
#print(soup.prettify())    #Commented out due to the extremely long page generated by it. 

Takes the wikipedia page and uses xml to find the table then checks to make sure the column width is uniform throughout.

In [164]:
doc = lh.fromstring(r.content)
mytable = doc.xpath('//tr')
[len(T) for T in mytable[:12]] #checks to see if every line has the right number of columns

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Takes the new table and finds the column headers

In [165]:
col = []
i=0

for t in mytable[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"


Sorts the data into the table 

In [166]:
for j in range(1,len(mytable)):
    T=mytable[j]
    
    if len(T)!=3:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content()
        
        col[i][1].append(data)
        i+=1

In [167]:
[len(C) for (title,C) in col] #check to make sure each column is the same length

[181, 181, 181]

Multiple steps: takes the new table and pulls it into a dataframe, drops off extra '\n' in the lines, gets rid of lines with 'Not assigned' and drops the final row which did not have data.

In [168]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df=df.replace(r'\n','',regex=True)
df.columns = ['Postal Code', 'Borough', 'Neighbourhood']
df = df[df.Borough != 'Not assigned']
df.drop(df.tail(1).index,inplace=True)


In [169]:
df.shape

(103, 3)